In [133]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
# Carregar os dados
dados = pd.read_parquet("dataset_com_clusters.parquet")
dados.columns


Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords', 'cluster', 'clusters_n'],
      dtype='object')

In [ ]:
import pandas as pd

def processar_dados(caminho_arquivo):
    # Carregar os dados
    dados = pd.read_parquet(caminho_arquivo)

    # Selecionar colunas relevantes
    dados = dados[["release_date", "revenue", "budget", "runtime", "genres", 
                  "original_language", "production_countries", "spoken_languages", "adult", "cluster", "clusters_n"]]

    # Filtrar dados (adult == False)
    dados = dados[dados["adult"] == False].copy()

    # Criar coluna 'disponibilidade_lucro'
    dados["disponibilidade_lucro"] = (
        dados["revenue"].apply(lambda x: isinstance(x, (int, float)) and x > 0) & 
        dados["budget"].apply(lambda x: isinstance(x, (int, float)) and x > 0)
    ).astype(int)

    # Eliminar registros com 'disponibilidade_lucro' igual a 0
    dados = dados[dados["disponibilidade_lucro"] == 1].copy()

    # Calcular 'lucro' e 'classificacao'
    dados['lucro'] = dados.apply(lambda row: row['revenue'] - row['budget']
                                 if isinstance(row['budget'], (int, float)) and isinstance(row['revenue'], (int, float))
                                 else None, axis=1)

    dados['classificacao'] = dados.apply(lambda row: 1 if row['lucro'] is not None and row['lucro'] and row['disponibilidade_lucro'] == 1 else 0, axis=1)

    # Criar o dataset 'dados_com_lucro' (classificacao == 1)
    dados_com_lucro = dados[dados['classificacao'] == 1]

    # Criar o dataset 'dados_sem_lucro' (classificacao == 0)
    dados_sem_lucro = dados[dados['classificacao'] == 0]

    # Criar duas amostras representativas de 'dados_com_lucro' (50% cada)
    metade_lucro_1 = dados_com_lucro.sample(frac=0.5, random_state=42)
    metade_lucro_2 = dados_com_lucro.drop(metade_lucro_1.index)  # O restante dos dados

    # Criar duas amostras representativas de 'dados_sem_lucro' (50% cada)
    metade_sem_lucro_1 = dados_sem_lucro.sample(frac=0.5, random_state=42)
    metade_sem_lucro_2 = dados_sem_lucro.drop(metade_sem_lucro_1.index)  # O restante dos dados

    # Concatenar as metades sem lucro com suas respectivas metades com lucro
    conjunto_1 = pd.concat([metade_lucro_1, metade_sem_lucro_1], ignore_index=True)
    conjunto_2 = pd.concat([metade_lucro_2, metade_sem_lucro_2], ignore_index=True)
    
    return conjunto_1, conjunto_2

# Exemplo de uso
dados_teste_balanceado, dados_treino_balanceado = processar_dados("dataset_com_clusters.parquet")

# Verificar o balanceamento
print(dados_teste_balanceado['classificacao'].value_counts(normalize=True))
print(dados_teste_balanceado['classificacao'].value_counts())

# Verificar o balanceamento
print(dados_treino_balanceado['classificacao'].value_counts(normalize=True))
print(dados_treino_balanceado['classificacao'].value_counts())


classificacao
1    0.961524
0    0.038476
Name: proportion, dtype: float64
classificacao
1    5248
0     210
Name: count, dtype: int64
classificacao
1    0.961517
0    0.038483
Name: proportion, dtype: float64
classificacao
1    5247
0     210
Name: count, dtype: int64


In [125]:

dados_teste = dados_teste_balanceado.copy()

dados_treino_filtrado = dados_treino_balanceado.copy()
# Criar novas colunas (número de idiomas e idade)
for df in [dados_treino_filtrado, dados_teste]:
    df["num_languages"] = df["spoken_languages"].apply(lambda x: len(x.split(",")) if isinstance(x, str) else 0)
    df["idade"] = df["release_date"].apply(lambda x: 2025 - int(x.split("-")[0]) if isinstance(x, str) else 0)

# Lista de gêneros
generos = ['Action', 'Science Fiction', 'Adventure', 'Drama', 'Crime',
           'Thriller', 'Fantasy', 'Comedy', 'Romance', 'Western', 'Mystery', 'War',
           'Animation', 'Family', 'Horror', 'Music']

def aplicar_one_hot_encoding_generos(df, generos):
    for genero in generos:
        df[genero] = df['genres'].apply(lambda x: 1 if isinstance(x, str) and genero in x.split(', ') else 0)
    return df

# Aplicar one-hot encoding de gêneros
dados_treino_filtrado = aplicar_one_hot_encoding_generos(dados_treino_filtrado, generos)
dados_teste = aplicar_one_hot_encoding_generos(dados_teste, generos)

def aplicar_one_hot_encoding_limitado(df, idiomas_permitidos, selected_countries):
    df['original_language_encoded'] = df['original_language']
    df['production_countries_encoded'] = df['production_countries']
    df['original_language_encoded'] = df['original_language_encoded'].apply(
        lambda x: x if x in idiomas_permitidos else 'other_language'
    )
    df['production_countries_encoded'] = df['production_countries_encoded'].apply(
        lambda x: x if x in selected_countries else 'other_country'
    )
    df = pd.get_dummies(df, columns=['original_language_encoded', 'production_countries_encoded'], prefix=['lang', 'country'])
    return df

# Listas de idiomas e países permitidos
idiomas_permitidos = ['en', 'fr', 'es', 'de', 'ja', 'zh', "pt", 'it']
selected_countries = ['United States', 'France', 'United Kingdom', 'Germany', 
                      'Canada', 'Japan', 'China', 'India', 'Italy', 'Spain']

# Aplicar one-hot encoding limitado
dados_treino_encoded = aplicar_one_hot_encoding_limitado(dados_treino_filtrado, idiomas_permitidos, selected_countries)
dados_teste_encoded = aplicar_one_hot_encoding_limitado(dados_teste, idiomas_permitidos, selected_countries)

# Garantir que a coluna 'genres' seja preservada
dados_treino_encoded['genres'] = dados_treino_filtrado['genres']
dados_teste_encoded['genres'] = dados_teste['genres']

# Alinhar colunas
dados_treino_encoded, dados_teste_encoded = dados_treino_encoded.align(dados_teste_encoded, join='outer', axis=1, fill_value=0)

dados_treino_encoded.to_parquet("dados_treino.parquet", index=False)
dados_teste_encoded.to_parquet("dados_teste.parquet", index=False)


In [ ]:
# Lista de colunas a serem removidas
colunas_para_remover = ["release_date", "revenue", "budget", "genres", "original_language", 
                        "production_countries", "spoken_languages", "adult", 
                        "disponibilidade_lucro", "lucro", "runtime", "idade"]

# Aplicar a remoção corretamente
for i, df in enumerate([dados_teste_encoded, dados_treino_encoded]):
    df = df.drop(columns=colunas_para_remover, errors='ignore')

    # Atribuir de volta ao DataFrame correto
    if i == 0:
        dados_teste_encoded = df
    else:
        dados_treino_encoded = df


dados_teste_encoded.columns

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import pandas as pd

# Separando features (X) e target (y)
X_train = dados_treino_encoded.drop(columns=['classificacao'])
y_train = dados_treino_encoded['classificacao']

X_test = dados_teste_encoded.drop(columns=['classificacao'])
y_test = dados_teste_encoded['classificacao']

# Criando e treinando o modelo SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')  # Kernel RBF padrão
svm_model.fit(X_train, y_train)

# Fazendo previsões
y_train_pred = svm_model.predict(X_train)
y_test_pred = svm_model.predict(X_test)

# Printando o classification report
print("Classification Report para Treino (SVM):")
print(classification_report(y_train, y_train_pred))

print("\nClassification Report para Teste (SVM):")
print(classification_report(y_test, y_test_pred))

In [128]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, recall_score

# Loop para encontrar o melhor valor de k com base no recall
melhor_k = None
melhor_recall = 0
for k in range(1, 21):  # Testando valores de k de 1 a 20
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(X_train, y_train)
    y_test_pred = knn_model.predict(X_test)
    recall = recall_score(y_test, y_test_pred)
    if recall > melhor_recall:
        melhor_recall = recall
        melhor_k = k

print(f"Melhor valor de k encontrado: {melhor_k}")
print(f"Melhor recall encontrado: {melhor_recall}")

# Treinando o modelo KNN com o melhor valor de k
knn_model = KNeighborsClassifier(n_neighbors=melhor_k)
knn_model.fit(X_train, y_train)

# Fazendo previsões com o melhor modelo
y_train_pred = knn_model.predict(X_train)
y_test_pred = knn_model.predict(X_test)

# Printando o classification report para o melhor modelo
print("\nClassification Report para Treino (KNN com melhor k):")
print(classification_report(y_train, y_train_pred))

print("\nClassification Report para Teste (KNN com melhor k):")
print(classification_report(y_test, y_test_pred))

Melhor valor de k encontrado: 3
Melhor recall encontrado: 0.9984756097560976

Classification Report para Treino (KNN com melhor k):
              precision    recall  f1-score   support

           0       0.89      0.20      0.32       210
           1       0.97      1.00      0.98      5247

    accuracy                           0.97      5457
   macro avg       0.93      0.60      0.65      5457
weighted avg       0.97      0.97      0.96      5457


Classification Report para Teste (KNN com melhor k):
              precision    recall  f1-score   support

           0       0.75      0.11      0.20       210
           1       0.97      1.00      0.98      5248

    accuracy                           0.96      5458
   macro avg       0.86      0.56      0.59      5458
weighted avg       0.96      0.96      0.95      5458



In [129]:
from sklearn.ensemble import RandomForestClassifier

melhor_n_estimators = None
melhor_recall = 0
for n_estimators in range(10, 110, 10):  # Testando valores de n_estimators de 10 a 100, pulando de 10 em 10
    rf_model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    rf_model.fit(X_train, y_train)
    y_test_pred = rf_model.predict(X_test)
    recall = recall_score(y_test, y_test_pred)
    if recall > melhor_recall:
        melhor_recall = recall
        melhor_n_estimators = n_estimators

print(f"Melhor valor de n_estimators encontrado: {melhor_n_estimators}")
print(f"Melhor recall encontrado: {melhor_recall}")

# Treinando o modelo Random Forest com o melhor valor de n_estimators
rf_model = RandomForestClassifier(n_estimators=melhor_n_estimators, random_state=42)
rf_model.fit(X_train, y_train)

# Fazendo previsões com o melhor modelo
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

# Printando o classification report para o melhor modelo
print("\nClassification Report para Treino (Random Forest com melhor n_estimators):")
print(classification_report(y_train, y_train_pred))

print("\nClassification Report para Teste (Random Forest com melhor n_estimators):")
print(classification_report(y_test, y_test_pred))

Melhor valor de n_estimators encontrado: 100
Melhor recall encontrado: 0.9929496951219512

Classification Report para Treino (Random Forest com melhor n_estimators):
              precision    recall  f1-score   support

           0       0.90      0.55      0.68       210
           1       0.98      1.00      0.99      5247

    accuracy                           0.98      5457
   macro avg       0.94      0.77      0.84      5457
weighted avg       0.98      0.98      0.98      5457


Classification Report para Teste (Random Forest com melhor n_estimators):
              precision    recall  f1-score   support

           0       0.53      0.20      0.29       210
           1       0.97      0.99      0.98      5248

    accuracy                           0.96      5458
   macro avg       0.75      0.60      0.64      5458
weighted avg       0.95      0.96      0.95      5458



In [130]:
import xgboost as xgb

# Loop para encontrar o melhor valor de n_estimators com base no recall
melhor_n_estimators = None
melhor_recall = 0
for n_estimators in range(10, 110, 10):  # Testando valores de n_estimators de 10 a 100, pulando de 10 em 10
    xgb_model = xgb.XGBClassifier(n_estimators=n_estimators, random_state=42)
    xgb_model.fit(X_train, y_train)
    y_test_pred = xgb_model.predict(X_test)
    recall = recall_score(y_test, y_test_pred)
    if recall > melhor_recall:
        melhor_recall = recall
        melhor_n_estimators = n_estimators

print(f"Melhor valor de n_estimators encontrado: {melhor_n_estimators}")
print(f"Melhor recall encontrado: {melhor_recall}")

# Treinando o modelo XGBoost com o melhor valor de n_estimators
xgb_model = xgb.XGBClassifier(n_estimators=melhor_n_estimators, random_state=42)
xgb_model.fit(X_train, y_train)

# Fazendo previsões com o melhor modelo
y_train_pred = xgb_model.predict(X_train)
y_test_pred = xgb_model.predict(X_test)

# Printando o classification report para o melhor modelo
print("\nClassification Report para Treino (XGBoost com melhor n_estimators):")
print(classification_report(y_train, y_train_pred))

print("\nClassification Report para Teste (XGBoost com melhor n_estimators):")
print(classification_report(y_test, y_test_pred))

Melhor valor de n_estimators encontrado: 10
Melhor recall encontrado: 0.9982850609756098

Classification Report para Treino (XGBoost com melhor n_estimators):
              precision    recall  f1-score   support

           0       0.88      0.18      0.30       210
           1       0.97      1.00      0.98      5247

    accuracy                           0.97      5457
   macro avg       0.93      0.59      0.64      5457
weighted avg       0.96      0.97      0.96      5457


Classification Report para Teste (XGBoost com melhor n_estimators):
              precision    recall  f1-score   support

           0       0.73      0.11      0.20       210
           1       0.97      1.00      0.98      5248

    accuracy                           0.96      5458
   macro avg       0.85      0.56      0.59      5458
weighted avg       0.96      0.96      0.95      5458



In [131]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import pandas as pd
import shap
from sklearn.model_selection import train_test_split

# Separando features (X) e target (y)
X_train = dados_treino_encoded.drop(columns=['classificacao'])
y_train = dados_treino_encoded['classificacao']

X_test = dados_teste_encoded.drop(columns=['classificacao'])
y_test = dados_teste_encoded['classificacao']

# Criando um sample representativo de 1% dos dados de treino
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=0.05, random_state=42)

# Criando e treinando o modelo SVM com o sample
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True)
svm_model.fit(X_train_sample, y_train_sample)

# Fazendo previsões com o sample
y_train_pred = svm_model.predict(X_train_sample)
y_test_pred = svm_model.predict(X_test)

# Printando o classification report
print("Classification Report para Treino (SVM com sample de 1%):")
print(classification_report(y_train_sample, y_train_pred))

print("\nClassification Report para Teste (SVM com sample de 1%):")
print(classification_report(y_test, y_test_pred))

# Aplicando o SHAP para explicar o modelo com o sample
explainer = shap.KernelExplainer(svm_model.predict_proba, X_train_sample)
shap_values = explainer.shap_values(X_train_sample)

# Selecionando os SHAP values para a classe positiva (classe 1)
shap_values_class_1 = shap_values[1]  # A classe 1 é geralmente a classe positiva

# Visualizando as importâncias das features com beeswarm plot
shap.summary_plot(shap_values_class_1, X_train_sample, feature_names=X_train_sample.columns, plot_type="dot")  # plot_type="dot" para beeswarm


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Using 272 background data samples could cause slower run times. Consider using shap.samp

Classification Report para Treino (SVM com sample de 1%):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.94      1.00      0.97       256

    accuracy                           0.94       272
   macro avg       0.47      0.50      0.48       272
weighted avg       0.89      0.94      0.91       272


Classification Report para Teste (SVM com sample de 1%):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       210
           1       0.96      1.00      0.98      5248

    accuracy                           0.96      5458
   macro avg       0.48      0.50      0.49      5458
weighted avg       0.92      0.96      0.94      5458



  2%|▏         | 5/272 [00:14<13:07,  2.95s/it]


KeyboardInterrupt: 